In [3]:
%matplotlib inline

from __future__ import absolute_import, print_function, unicode_literals, division
from sklearn.datasets import fetch_mldata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
import random as rand
import copy

In [4]:
def mat_dbg(x):
    print(x.shape, ": \n", x)

In [50]:
class MLP(object):
    """
    NOTE: Matrix operations are modified from RBM file
    In particular, we have an an an input of n data points with
    dimension d as a d-by-n matrix
    """
    def __init__(self, data, num_hidden, learn_rate):
        # First row are the target values
        self.targets = data[0]
        self.data = data[1:,:]
        self.num_data = data.shape[1]
        self.num_visible = data.shape[0] - 1
        self.num_hidden = num_hidden
        self.learn_rate = learn_rate
        self.state_size = 2 * (np.amax(self.targets) - np.amin(self.targets)) + 4

        self.weights = 2 * np.random.rand(self.num_visible + 1, self.num_hidden) - 1
        self.data = np.insert(self.data, 0, 1, axis = 0)
    
    def _sigmoid(self,x):
        return 1.0/(1.0+np.exp(-x))
        
    def _activated(self,mat):
        activated = self._sigmoid(np.dot(self.weights.T,mat))
        return activated
    
    def _gradient(self,mat):
        w = copy.deepcopy(mat)
        for elt in np.nditer(w,op_flags=['readwrite']):
            elt[...] = elt - elt * elt
        return w
    
    def train(self, iterations=3):
        data_copy = copy.deepcopy(self.data)
        for j in range(0,iterations):
            for i in range(0,self.num_data):

                """Feed-forward phase"""
                data_col = data_copy[:,i]
                data_col = np.reshape(data_col, (-1, 1))

                activated_output = (self._activated(data_col))[0]
                guess = self.state_size * (2 * activated_output - 1)

                """Error calculation phase"""
                error = self.targets[i] - guess

                """Back-propagation phase"""
                grad = (self._gradient(activated_output))[0]

                c = (error * self.learn_rate)[0]
                wt_change = c * grad * data_col

                self.weights = wt_change + self.weights
    
    def pred(self, test):
        test_copy = copy.deepcopy(test)
        test_copy = np.insert(test_copy, 0, 1, axis = 0)
        pred_p = self._activated(test_copy)
        pred = 2 * pred_p - 1
        pred = self.state_size * pred
        return pred


In [83]:
# See www.cse.unsw.edu.au/~cs9417ml/MLP2/
# See https://www.hiit.fi/u/ahonkela/dippa/node41.html

# Generate training data (no noise)
dims = 10
data_pts = 300 * dims
sample_data = np.sign(2*np.random.rand(dims,data_pts)-1)
sample_data[0] = (4 * sample_data[1] ** 2 + 1 * sample_data[2]) 
# Need to write binary target values for data as well for MLP
# mat_dbg(sample_data)
MLP_1 = MLP(sample_data,1,0.01) #0.035 threshold
print("Weights^T: \n",MLP_1.weights.T)
print("Targets: \n",MLP_1.targets)
print("Data sample: \n",MLP_1.data[1:5])

Weights^T: 
 [[-0.91796034  0.9163448  -0.37576023 -0.06045506  0.68782146  0.28630357
  -0.76020405  0.92430764  0.78151779 -0.8989808 ]]
Targets: 
 [ 5.  3.  3. ...,  5.  5.  3.]
Data sample: 
 [[ 1.  1.  1. ..., -1.  1. -1.]
 [ 1. -1. -1. ...,  1.  1. -1.]
 [ 1.  1.  1. ...,  1. -1. -1.]
 [-1.  1.  1. ..., -1. -1.  1.]]


In [84]:
MLP_1.train(5)
print("Weights: \n", MLP_1.weights)
print("Targets: \n", MLP_1.targets)

Weights: 
 [[  1.12739721e+00]
 [  7.67541712e-16]
 [  3.38939854e-01]
 [  4.23928828e-16]
 [ -7.49775764e-17]
 [  6.91097314e-16]
 [ -6.61724269e-16]
 [ -4.82791299e-16]
 [ -1.93727390e-16]
 [  7.68328969e-16]]
Targets: 
 [ 5.  3.  3. ...,  5.  5.  3.]


In [85]:
dims = 9
test_pts = 10
test_data = np.sign(2*np.random.rand(dims,test_pts)-1)
print("Test data: \n",test_data)
print("Shape:",np.shape(a))
print("targets: \n", 4 * test_data[0] ** 2 + 1 * test_data[1] )
guesses = MLP_1.pred(test_data)
print("Guesses: \n", guesses)

Test data: 
 [[ 1. -1. -1.  1. -1. -1.  1.  1. -1. -1.]
 [ 1.  1. -1.  1.  1.  1.  1. -1. -1.  1.]
 [-1. -1.  1. -1. -1.  1. -1. -1. -1. -1.]
 [-1.  1.  1. -1. -1. -1.  1. -1.  1. -1.]
 [-1. -1. -1.  1.  1.  1. -1. -1.  1. -1.]
 [ 1. -1.  1. -1. -1.  1.  1.  1. -1.  1.]
 [ 1. -1. -1.  1.  1.  1.  1.  1.  1. -1.]
 [ 1. -1.  1. -1. -1. -1. -1.  1.  1.  1.]
 [-1. -1. -1.  1.  1. -1.  1.  1. -1. -1.]]
Shape: (1, 10)
targets: 
 [ 5.  5.  3.  5.  5.  5.  5.  3.  3.  5.]
Guesses: 
 [[ 5.  5.  3.  5.  5.  5.  5.  3.  3.  5.]]


In [86]:
30 / 7 / 75

0.05714285714285714